# Heating costs model

In [ ]:
import pandas as pd
import numpy as np
import math
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score, mean_absolute_error

In [ ]:
#!wget https://raw.githubusercontent.com/magnuspaal/germany-rental-ml/master/data/rental/rental_location_data.csv -O rental_location_data.csv
!wget https://github.com/magnuspaal/germany-rental-ml/raw/master/data/rental/rental_location_data_nan.csv -O rental_location_data.csv

--2020-12-14 15:44:58--  https://github.com/magnuspaal/germany-rental-ml/raw/master/data/rental/rental_location_data_nan.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/magnuspaal/germany-rental-ml/master/data/rental/rental_location_data_nan.csv [following]
--2020-12-14 15:44:58--  https://raw.githubusercontent.com/magnuspaal/germany-rental-ml/master/data/rental/rental_location_data_nan.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46950277 (45M) [text/plain]
Saving to: ‘rental_location_data.csv’

rental_location_dat 100%[===================>]  44.77M  71.1MB/s    in 0.6s    

2020-12-14 15

In [ ]:
rental_location_data = pd.read_csv("rental_location_data.csv")

#drop index column
rental_location_data = rental_location_data.drop(rental_location_data.columns[0], axis=1)
assert rental_location_data.shape[1] == 44

In [ ]:
# remove the biggest outliers in terms of totalrent
print("rows before", rental_location_data.shape[0])
rental_location_data = rental_location_data[(rental_location_data['baseRent'] < 1500)]
rental_location_data = rental_location_data[(rental_location_data['totalRent'] < 2500)]
rental_location_data = rental_location_data.dropna(subset=['heatingCosts'])
print("rows after", rental_location_data.shape[0])

rows before 186793
rows after 49979


In [ ]:
# remove the filled in heating data - helps model generalize
# 95th percentile of heating costs
rental_location_data_without_nafill = rental_location_data[(rental_location_data['heatingCosts'] <= 125) &
                                                           (rental_location_data['heatingCosts'] != 0.0)]

In [ ]:
def print_metrics(model, X_train, y_train, X_test, y_test):
  train_predictions = model.predict(X_train).reshape(-1, 1)
  test_predictions = model.predict(X_test).reshape(-1, 1)

  print("xgb score on train",model.score(X_train, y_train))
  print("xgb score on test", model.score(X_test, y_test))
  print("mse on train", math.sqrt(mean_squared_error(y_train, train_predictions)))
  print("mse on test", math.sqrt(mean_squared_error(y_test, test_predictions.reshape(-1, 1))))
  print("mas on train", mean_absolute_error(y_train, train_predictions))
  print("mas on test", mean_absolute_error(y_test, test_predictions.reshape(-1, 1)))
  print("r2 score on train", r2_score(y_train, train_predictions))
  print("r2 score on test", r2_score(y_test, test_predictions.reshape(-1, 1)))

# Trying with different approaches here

In [ ]:
X = rental_location_data_without_nafill[['yearConstructed', 'baseRent', 'floor', 'noRooms', 'heatingType', 'lat']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

model_xgb = xgb.XGBRFRegressor(random_state=0, n_estimators=600, max_depth=50)
model_xgb.fit(X_train, y_train)


print_metrics(model_xgb, X_train, y_train, X_test, y_test)

[15:45:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.3769637167820493
xgb score on test 0.3479572703839826
mse on train 18.03154061832406
mse on test 18.609972107903094
mas on train 14.128416787763568
mas on test 14.598120592339408
r2 score on train 0.3769637167820493
r2 score on test 0.3479572703839826


In [ ]:
X = rental_location_data_without_nafill[['typeOfFlat', 'yearConstructed', 'baseRent', 'floor', 'noRooms', 'heatingType', 'lat', 'livingSpace']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

model_xgb_rf_livingspace = xgb.XGBRFRegressor(random_state=0, n_estimators=600, max_depth=50)
model_xgb_rf_livingspace.fit(X_train, y_train)


print_metrics(model_xgb_rf_livingspace, X_train, y_train, X_test, y_test)

[15:46:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.4662498589195788
xgb score on test 0.4276002441245732
mse on train 16.689570730594426
mse on test 17.43642370172872
mas on train 12.842059835977599
mas on test 13.386168183582857
r2 score on train 0.4662498589195788
r2 score on test 0.4276002441245732


In [ ]:
X = rental_location_data_without_nafill[['typeOfFlat', 'yearConstructed', 'baseRent', 'floor', 'noRooms', 'heatingType', 'energyEfficiencyClass', 'lat', 'livingSpace']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

model_xgb_rf_efficiency = xgb.XGBRFRegressor(random_state=0, n_estimators=600, max_depth=50)
model_xgb_rf_efficiency.fit(X_train, y_train)


print_metrics(model_xgb_rf_efficiency, X_train, y_train, X_test, y_test)

[15:47:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.47375767233603266
xgb score on test 0.4312827173649996
mse on train 16.57177597053565
mse on test 17.380245508711464
mas on train 12.755658802506439
mas on test 13.336520985982236
r2 score on train 0.4737576723360327
r2 score on test 0.43128271736499957


In [ ]:
X = rental_location_data_without_nafill[['typeOfFlat', 'yearConstructed', 'baseRent', 'floor', 'noRooms', 'heatingType', 'energyEfficiencyClass', 'lat', 'lon', 'livingSpace']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()
X['lon'] = X['lon'] - X['lon'].mean()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_xgb_rf_latlon = xgb.XGBRFRegressor(random_state=0, n_estimators=300, max_depth=30)
model_xgb_rf_latlon.fit(X_train, y_train)


print_metrics(model_xgb_rf_latlon, X_train, y_train, X_test, y_test)

[15:49:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.48278602850208957
xgb score on test 0.4404940811104451
mse on train 16.443594439953717
mse on test 17.140947843540793
mas on train 12.679280367273737
mas on test 13.144587837466158
r2 score on train 0.4827860285020896
r2 score on test 0.4404940811104451


In [148]:
X = rental_location_data_without_nafill[['typeOfFlat', 'yearConstructed', 'baseRent', 'floor', 'noRooms', 'heatingType', 'energyEfficiencyClass', 'lat', 'lon', 'livingSpace']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()
X['lon'] = X['lon'] - X['lon'].mean()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model_xgb_regr = xgb.XGBRegressor(random_state=0, n_estimators=300, max_depth=6)
model_xgb_regr.fit(X_train, y_train)


print_metrics(model_xgb_regr, X_train, y_train, X_test, y_test)

[16:05:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.6464478218135561
xgb score on test 0.5054986131635345
mse on train 13.58887189361198
mse on test 16.12021810173923
mas on train 10.244310653168506
mas on test 12.096535001445613
r2 score on train 0.6464478218135561
r2 score on test 0.5054986131635345


In [ ]:
print('mean and std dev of heating costs of test', y.mean(), y.std())

mean and std dev of heating costs of test 69.97461172185312 22.87505688528213


In [ ]:
X = rental_location_data_without_nafill[['typeOfFlat', 'yearConstructed', 'regio1', 'baseRent', 'floor', 'noRooms', 'heatingType', 'energyEfficiencyClass', 'lat', 'lon', 'livingSpace']]
y = rental_location_data_without_nafill['heatingCosts']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()
X['lon'] = X['lon'] - X['lon'].mean()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_xgb_regr = xgb.XGBRFRegressor(random_state=0, n_estimators=300, max_depth=20)
model_xgb_regr.fit(X_train, y_train)


print_metrics(model_xgb_regr, X_train, y_train, X_test, y_test)

[15:50:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.4849355819164119
xgb score on test 0.44203981177985563
mse on train 16.4093888808069
mse on test 17.117254069577893
mas on train 12.650892019583132
mas on test 13.124542444033473
r2 score on train 0.48493558191641184
r2 score on test 0.44203981177985563


# CO2 estimation

https://github.com/tmrowco/bloom-contrib/blob/master/co2eq/energy/energyfootprints.yml


Current approach: 


In [ ]:
rental_location_data_without_nafill['heatingType'].value_counts()

central_heating                   24992
district_heating                   6216
gas_heating                        2691
floor_heating                      1502
self_contained_central_heating     1134
oil_heating                         597
combined_heat_and_power_plant       250
heat_pump                           215
wood_pellet_heating                 108
night_storage_heater                 66
electric_heating                     38
solar_heating                        19
stove_heating                        16
Name: heatingType, dtype: int64

In [ ]:
# https://nottenergy.com/resources/energy-cost-comparison/

def get_cost_per_kwh(source):
  # default is gas (most popular in germany) https://www.statista.com/statistics/1189752/household-heating-sources-germany/
  if (source == 'central_heating' or source == 'district_heating'):
    return 0.21868
  if (source == 'gas_heating' or source == 'floor_heating' or source == 'self_contained_central_heating'):
    return 0.21868
  if (source == 'oil_heating'):
    return 0.0035
  if (source == 'wood_pellet_heating'):
    return 0.068
  if (source == 'solar_heating'):
    return 0.098
  else:
    return 0.21868

# https://github.com/tmrowco/bloom-contrib/blob/master/co2eq/energy/energyfootprints.yml
def get_co2kg_per_kwh(source):
  if (source == 'central_heating' or source == 'district_heating'):
    return 0.295
  if (source == 'gas_heating' or source == 'floor_heating' or source == 'self_contained_central_heating'):
    return 0.295
  if (source == 'oil_heating'):
    return 0.43
  if (source == 'wood_pellet_heating'):
    return 0.1
  if (source == 'solar_heating'):
    return 0.0225
  else:
    return 0.295

def costs_to_co2_kg(source, cost):
  kwh_used = cost / get_cost_per_kwh(source)
  co2_generated = kwh_used * get_co2kg_per_kwh(source)
  return co2_generated

heating_costs_pred = model_xgb_regr.predict(X_test)
heating_costs_real = y_test.to_numpy()
heating_sources = X_test.loc[:, X_test.columns.str.startswith('heatingType')].idxmax(axis=1).str.lstrip('heatingType_')

heating_sources_costs_pred = pd.DataFrame({'heating_type': heating_sources, 'heating_costs': heating_costs_pred })
heating_sources_costs_real = pd.DataFrame({'heating_type': heating_sources, 'heating_costs': heating_costs_real })

heating_sources_costs_pred['heating_co2kg_monthly'] = heating_sources_costs_pred.apply(lambda row: costs_to_co2_kg(row['heating_type'], row['heating_costs']), axis=1)
heating_sources_costs_real['heating_co2kg_monthly'] = heating_sources_costs_real.apply(lambda row: costs_to_co2_kg(row['heating_type'], row['heating_costs']), axis=1)


heating_sources_costs_real

,heating_type,heating_costs,heating_co2kg_monthly
48401,central_heating,50.00,67.450155
106805,central_heating,90.00,121.410280
32416,floor_heating,33.38,45.029724
115386,central_heating,100.00,134.900311
70372,central_heating,113.00,152.437351
...,...,...,...
114970,central_heating,50.00,67.450155
167028,central_heating,70.00,94.430218
120542,district_heating,33.00,44.517103
138181,central_heating,60.00,80.940187


In [ ]:
rental_location_data_with_co2 = rental_location_data_without_nafill.copy()

rental_location_data_with_co2['heatingCo2'] = rental_location_data_with_co2.apply(lambda row: costs_to_co2_kg(row['heatingType'], row['heatingCosts']), axis=1)
rental_location_data_with_co2



,regio1,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,cellar,yearConstructedRange,baseRent,livingSpace,condition,interiorQual,petsAllowed,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,city,zip,lat,lon,heatingCo2
3,Bremen,98.00,central_heating,ONE_YEAR_FREE,0,1,7,3.22,40.0,722.26,1962.0,113252899,NaN,gas,0,1,2.0,542.26,57.08,NaN,NaN,NaN,0,4,ground_floor,28213,2.0,75.0,NaN,NaN,2,0,2,Bremen,Riensberg,82.0,C,NaN,NaN,NaN,Bremen,28213,53.075160,8.807770,110.618255
8,Bremen,62.00,central_heating,ONE_YEAR_FREE,0,0,8,5.32,40.0,746.00,NaN,105962333,NaN,NaN,0,0,NaN,624.00,58.05,NaN,NaN,negotiable,0,5,NaN,28213,2.0,NaN,2.0,NaN,2,0,2,Bremen,Neu_Schwachhausen,60.0,NaN,NaN,NaN,NaN,Bremen,28213,53.075160,8.807770,80.940187
9,Bremen,150.00,NaN,ONE_YEAR_FREE,0,1,12,2.99,40.0,975.00,1979.0,85131797,NaN,oil,1,1,3.0,700.00,78.00,well_kept,NaN,NaN,0,5,apartment,28213,3.0,121.0,1.0,NaN,3,0,3,Bremen,Riensberg,125.0,NO_INFORMATION,NaN,NaN,NaN,Bremen,28213,53.075160,8.807770,168.625389
16,Bremen,108.00,central_heating,ONE_YEAR_FREE,0,1,9,2.99,40.0,781.20,1961.0,114698981,NaN,district_heating,0,1,2.0,583.20,62.71,NaN,NaN,NaN,0,4,apartment,28213,2.0,101.0,2.0,NaN,2,0,3,Bremen,Riensberg,90.0,D,NaN,NaN,NaN,Bremen,28213,53.075160,8.807770,121.410280
17,Bremen,87.00,central_heating,ONE_YEAR_FREE,0,1,4,3.22,40.0,780.78,1961.0,112984781,NaN,district_heating,0,1,2.0,571.78,59.56,NaN,NaN,NaN,0,4,apartment,28213,2.0,172.0,1.0,NaN,2,0,2,Bremen,Riensberg,122.0,F,NaN,NaN,NaN,Bremen,28213,53.075160,8.807770,164.578379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186766,Mecklenburg_Vorpommern,60.00,central_heating,NaN,0,0,2,5.13,NaN,297.66,1960.0,104342494,NaN,gas,0,1,2.0,188.66,33.69,NaN,NaN,negotiable,0,1,apartment,19217,1.0,160.0,1.0,NaN,1,0,1,Nordwestmecklenburg_Kreis,Groß_Molzahn,49.0,NO_INFORMATION,NaN,NaN,NaN,Groß_Molzahn,19217,53.735220,10.874210,66.101152
186770,Schleswig_Holstein,100.00,central_heating,ONE_YEAR_FREE,0,1,35,0.00,40.0,540.00,NaN,104763581,1.0,pellet_heating:gas,1,0,NaN,350.00,74.00,well_kept,normal,no,0,2,apartment,24996,3.5,NaN,0.0,2.0,3,1,3,Schleswig_Flensburg_Kreis,Sterup,90.0,NaN,NaN,71.43,0.2205,Sterup,24996,54.727785,9.739278,121.410280
186773,Thüringen,30.00,central_heating,NONE,0,1,4,2.00,2.4,290.00,1980.0,94380257,NaN,gas,0,1,3.0,210.00,38.00,modernized,normal,yes,0,1,apartment,99310,1.0,NaN,0.0,2.0,1,0,1,Ilm_Kreis,Wipfratal,50.0,NaN,2017.0,NaN,NaN,Wipfratal,99310,50.783300,11.000000,67.450155
186775,Mecklenburg_Vorpommern,30.00,NaN,NaN,0,0,11,2.56,NaN,290.00,NaN,105494302,NaN,NaN,0,1,NaN,195.00,53.77,fully_renovated,NaN,NaN,0,1,NaN,17390,2.0,NaN,NaN,NaN,2,0,2,Ostvorpommern_Kreis,Rubkow,65.0,NaN,NaN,71.43,0.2276,Rubkow,17390,53.933620,13.720010,87.685202


In [ ]:
# test co2 model
X = rental_location_data_with_co2[['typeOfFlat', 'yearConstructed', 'regio1', 'baseRent', 'floor', 'noRooms', 'heatingType', 'energyEfficiencyClass', 'lat', 'lon', 'livingSpace']]
y = rental_location_data_with_co2['heatingCo2']

X = pd.get_dummies(X)
X['yearConstructed'] = X['yearConstructed'] - X['yearConstructed'].mean()
X['baseRent'] = (X['baseRent'] - X['baseRent'].min())/(X['baseRent'].max() - X['baseRent'].min())
X['lat'] = X['lat'] - X['lat'].mean()
X['lon'] = X['lon'] - X['lon'].mean()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_xgb_co2 = xgb.XGBRFRegressor(random_state=0, n_estimators=300, max_depth=20)
model_xgb_co2.fit(X_train, y_train)


print_metrics(model_xgb_co2, X_train, y_train, X_test, y_test)

[15:51:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgb score on train 0.9483738011147896
xgb score on test 0.9483339638584681
mse on train 251.0135351230964
mse on test 248.04481093890703
mas on train 39.098085931341856
mas on test 38.98522934122561
r2 score on train 0.9483738011147896
r2 score on test 0.948333963858468
